In [87]:
# %pip install pandas scikit-learn scikit-learn-intelex

In [88]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
import pandas as pd
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [89]:
data = pd.read_csv("dataset/train.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24590 entries, 0 to 24589
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   row_id    22297 non-null  float64
 1   date      22297 non-null  object 
 2   state     22297 non-null  object 
 3   store     22297 non-null  object 
 4   product   22297 non-null  object 
 5   num_sold  22297 non-null  float64
dtypes: float64(2), object(4)
memory usage: 1.1+ MB


In [90]:
data.dropna(inplace=True)

In [91]:
X = data.drop('num_sold', axis=1)
Y = data['num_sold']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [92]:
categorical_features = ['state', 'store', 'product']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ])


In [93]:
rf_regressor = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RandomForestRegressor(random_state=0))])

In [94]:
rf_regressor.fit(X_train, Y_train)
Y_pred = rf_regressor.predict(X_test)

In [95]:
mse = mean_squared_error(Y_test, Y_pred)
print("Mean Squared Error:", mse)

r2 = r2_score(Y_test, Y_pred)
print("R2 Score:", r2)

Mean Squared Error: 13561.228502167987
R2 Score: 0.8072780848646062
